In [ ]:
import matplotlib.pyplot as plt
import logging
from autoemulate.experimental.emulators.the_well import TheWellFNO
from autoemulate.experimental.data.spatiotemporal_dataset import AdvectionDiffusionDataset, AutoEmulateDataModule

In [ ]:
# Make an autoemulate datamodule from the_well datamodule
from autoemulate.simulations.advection_diffusion import AdvectionDiffusion
rd = AdvectionDiffusion(n=64, T=10, dt=0.1, return_timeseries=True)
data = rd.forward_samples_spatiotemporal(200)
# data = rd.forward_samples_spatiotemporal(20)
y = data["data"]
data_valid = rd.forward_samples_spatiotemporal(4)
data_test = rd.forward_samples_spatiotemporal(4)

In [ ]:
logging.basicConfig(level=logging.INFO)

ae_data_module = AutoEmulateDataModule(
    n_steps_input=4,
    n_steps_output=10,
    data_path=None,
    dataset_cls=AdvectionDiffusionDataset,
    data={"train": data, "valid": data_valid, "test": data_test},
    verbose=False
)
output_path = "../data/the_well/runs/advection_diffusion_wip_fix_2"

In [ ]:
# Plot example
batch = next(iter(ae_data_module.val_dataloader()))
plt.imshow(batch["input_fields"][0, 0, :, :, 0])
plt.show()

In [ ]:
# Initialize the emulator
from the_well.data.data_formatter import DefaultChannelsFirstFormatter
from the_well.benchmark.metrics import VRMSE
from autoemulate.experimental.emulators.the_well import TheWellAFNO, TheWellFNOWithTime, TrainerParams, TheWellFNOWithLearnableWeights, DefaultChannelsFirstFormatterWithTime

# FNO with learnable weights
# em = TheWellFNOWithLearnableWeights(
#     datamodule=ae_data_module,
#     formatter_cls=DefaultChannelsFirstFormatter,
#     loss_fn=VRMSE(),
#     trainer_params=TrainerParams(
#         device="mps",
#         output_path=output_path,
#         max_rollout_steps=100,
#         optimizer_params={"lr": 1e-5}
#     )
# )

# FNO with teacher forcing
em = TheWellFNO(
    datamodule=ae_data_module,
    formatter_cls=DefaultChannelsFirstFormatter,
    loss_fn=VRMSE(),
    trainer_params=TrainerParams(
        device="mps",
        output_path=output_path,
        max_rollout_steps=100,
        optimizer_params={"lr": 1e-3},
        enable_tf_schedule=True,
    )
)

# FNO with time
em = TheWellFNOWithTime(
    datamodule=ae_data_module,
    formatter_cls=DefaultChannelsFirstFormatterWithTime,
    loss_fn=VRMSE(),
    modes_time=2,
    modes1=16,
    modes2=16,
    trainer_params=TrainerParams(
        device="mps",
        output_path=output_path,
        max_rollout_steps=100,
        optimizer_params={"lr": 1e-2}
    )
)

In [ ]:
# Example of different rollouts
y_true = batch["output_fields"].to(em.device)
y_teacher_forcing = em.trainer.rollout_model(em.model, batch, em.trainer.formatter, train=False, teacher_forcing=True)[0]
y_free_running = em.trainer.rollout_model(em.model, batch, em.trainer.formatter, train=False, teacher_forcing=False)[0]

In [ ]:
# Explore passing predictions to loss
# vrmse = VRMSE()
# print("Teacher forcing VRMSE:", vrmse(y_teacher_forcing, y_true, ae_data_module.train_dataset.metadata).detach().cpu())
# print("Free running VRMSE:", vrmse(y_free_running, y_true, ae_data_module.train_dataset.metadata).detach().cpu()[:1])

In [ ]:
# Fit the model
em.fit()

In [ ]:
# Validation loop
valid_results = em.trainer.validation_loop(
    ae_data_module.rollout_val_dataloader(),
    valid_or_test="rollout_valid",
    full=True
)

In [ ]:
test_results = em.trainer.validation_loop(
    ae_data_module.rollout_test_dataloader(),
    valid_or_test="rollout_test",
    full=True
)

In [ ]:
from pprint import pprint
pprint(valid_results)

In [ ]:
pprint(test_results)

In [ ]:
# Run prediction from a dataloader
em.predict(ae_data_module.rollout_test_dataloader()).shape

In [ ]:
# Run prediction from a non-rollout dataloader
em.predict(ae_data_module.test_dataloader()).shape


In [ ]:
# Initialize a UNet emulator
from autoemulate.experimental.emulators.the_well import TheWellUNetClassic


em = TheWellUNetClassic(datamodule=ae_data_module, output_path=output_path, device="cpu")

In [ ]:
em.fit()

In [ ]:
em.predict(ae_data_module.rollout_test_dataloader()).shape